# Transform Orders Data - Explode Arrays
1. Access elements from the JSON object
2. Deduplicate Array elements
3. Explode Arrays
4. Write the Transformed Data to Silver Schema

In [0]:
select * 
from gizmobox.silver.orders_json; 

### 1. Access elements from the JSON object
column_name.object

In [0]:
select json_value.order_id,
json_value.order_status,
json_value.payment_method,
json_value.total_amount,
json_value.transaction_timestamp,
json_value.customer_id,
json_value.items
from gizmobox.silver.orders_json;

### 2. Deduplicate Array elements
array_distinct function

In [0]:
select json_value.order_id,
json_value.order_status,
json_value.payment_method,
json_value.total_amount,
json_value.transaction_timestamp,
json_value.customer_id,
array_distinct(json_value.items) as items
from gizmobox.silver.orders_json;

### 3. Explode Arrays
explode function

In [0]:
select json_value.order_id,
        json_value.order_status,
        json_value.payment_method,
        json_value.total_amount,
        json_value.transaction_timestamp,
        json_value.customer_id,
        explode(array_distinct(json_value.items)) as item
from gizmobox.silver.orders_json;

In [0]:
create or replace temporary view tv_orders_exploded
as
select json_value.order_id,
        json_value.order_status,
        json_value.payment_method,
        json_value.total_amount,
        json_value.transaction_timestamp,
        json_value.customer_id,
        explode(array_distinct(json_value.items)) as item
from gizmobox.silver.orders_json;

In [0]:
select order_id,
       order_status,
       payment_method,
       total_amount,
       transaction_timestamp,
       customer_id,
       item.item_id,
       item.name,
       item.price,
       item.quantity,
       item.category,
       item.details.brand,
       item.details.color
from tv_orders_exploded;

### 4. Write the Transformed Data to Silver Schema

In [0]:
create table if not exists gizmobox.silver.orders
as
select order_id,
       order_status,
       payment_method,
       total_amount,
       transaction_timestamp,
       customer_id,
       item.item_id,
       item.name,
       item.price,
       item.quantity,
       item.category,
       item.details.brand,
       item.details.color
from tv_orders_exploded;

In [0]:
select *
from gizmobox.silver.orders;